In [ ]:
# !nvidia-smi

Thu Sep  7 04:57:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### install dependencies

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.32.1 --progress-bar off
!pip install -qqq datasets==2.14.4 --progress-bar off
!pip install -qqq peft==0.5.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off
!pip install -qqq trl==0.7.1 --progress-bar off

### Importing libraries

In [ ]:
import pandas as pd
import pprint
import json
import re

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, TrainingArguments
from huggingface_hub import notebook_login
from datasets import load_dataset, Dataset
from peft import LoraConfig, PeftModel

from trl import SFTTrainer

### Load dataset

In [ ]:
dataset = load_dataset('Salesforce/dialogstudio', 'TweetSumm')
dataset

DatasetDict({
    train: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 879
    })
    validation: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
    test: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
})

### Preparing text and prompting

In [ ]:
# Training Prompt
DEFAULT_SYS_PROMPT = """
Below is a conversation between a human and an AI agent. Write a summary of the conversation.
""".strip()

def generate_training_prompt(
    conversation:str, summary:str, system_prompt:str=DEFAULT_SYS_PROMPT
) -> str :
  return f"""### Instruction: {system_prompt}

### Input:
{conversation.strip()}

### Response:
{summary}
""".strip()

In [ ]:
# Preprocess Text
def clean_text(text) :
  text = re.sub(r"http\+S", "", text) # removing URLs followed by non white spaces
  text = re.sub(r"@[^\s]+", "", text) # removing mentions and usernames
  text = re.sub(r"\s+", " ", text) # replacing consecutive white spaces with only one space
  return re.sub(r"\^[^ ]+", "", text)

In [ ]:
def create_conversation(data_point) :

  text=''
  for item in data_point['log'] :
    # Preprocess the user input
    user = clean_text(item['user utterance'])
    # adding dialoge to text
    text += f"user: {user.strip()}\n"

    # Preprocess system response
    agent = clean_text(item["system response"])
    # adding to text
    text += f"agent: {agent.strip()}\n"

    # return text which contains input summart pairs
    return text


def generate_text(data_point) :
  '''
  input :
    data_point : is the dialoges of the training data
  '''
  summaries = json.loads(data_point['original dialog info'])['summaries']['abstractive_summaries']
  summary = summaries[0]
  summary = " ".join(summary)

  conversation_text = create_conversation(data_point)

  return {
      "conversation" : conversation_text,
      "summary" : summary,
      'text' : generate_training_prompt(conversation_text, summary)
  }

In [ ]:
example = generate_text(dataset['train'][0])
print(example["text"])


### Instruction: Below is a conversation between a human and an AI agent. Write a summary of the conversation.

### Input:
user: So neither my iPhone nor my Apple Watch are recording my steps/activity, and Health doesn’t recognise either source anymore for some reason. Any ideas? https://t.co/m9DPQbkftD please read the above.
agent: Let’s investigate this together. To start, can you tell us the software versions your iPhone and Apple Watch are running currently?

### Response:
Customer enquired about his Iphone and Apple watch which is not showing his any steps/activity and health activities. Agent is asking to move to DM and look into it.


### Processing the whole dataset

In [ ]:
def process_dataset(data:dataset) :
  return (
      data.shuffle(seed=42).map(generate_text).remove_columns(
          [
              "original dialog id",
                "new dialog id",
                "dialog index",
                "original dialog info",
                "log",
                "prompt",
          ]
      )
  )

In [ ]:
dataset["train"] = process_dataset(dataset["train"])
dataset["validation"] = process_dataset(dataset["validation"])
dataset

DatasetDict({
    train: Dataset({
        features: ['conversation', 'summary', 'text'],
        num_rows: 879
    })
    validation: Dataset({
        features: ['conversation', 'summary', 'text'],
        num_rows: 110
    })
    test: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
})

### Model + Tokenization + Configiration

In [ ]:
# You should get a permission for using llama 2 from both hugging face and META
notebook_login()

In [ ]:
model_name = "meta-llama/Llama-2-7b-hf"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    use_safetensors = True,
    quantization_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_quant_type = "nf4",
        bnb_4bit_compute_dtype = torch.float16,
    ),
    trust_remote_code = True,
    device_map = 'auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [ ]:
MODEL_NAME = 'meta-llama/Llama-2-7b-hf'
def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer


model, tokenizer = create_model_and_tokenizer()
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 'load_in_8bit': False,
 'load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float16'}

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha = 64,
    lora_dropout = 0.1,
    target_modules = [
        "q_proj",
        "up_proj",
        "o_proj",
        "k_proj",
        "down_proj",
        "gate_proj",
        "v_proj",
],
    bias = "none",
    task_type = "CAUSAL_LM",
)

In [ ]:
OUTPUT_DIR = "experiments"
%load_ext tensorboard
%tensorboard --logdir experiments/runs

In [ ]:
training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
22,1.895700,1.844426
44,1.803900,1.794598
66,1.515800,1.774863
88,1.692200,1.768136
110,1.602100,1.766801


TrainOutput(global_step=110, training_loss=1.791326518492265, metrics={'train_runtime': 1553.2605, 'train_samples_per_second': 1.132, 'train_steps_per_second': 0.071, 'total_flos': 5739988963958784.0, 'train_loss': 1.791326518492265, 'epoch': 2.0})

In [ ]:
trainer.save_model()

In [ ]:
trainer.model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
              

In [ ]:
from peft import AutoPeftModelForCausalLM

trained_model = AutoPeftModelForCausalLM.from_pretrained(
    OUTPUT_DIR,
    low_cpu_mem_usage=True,
)

merged_model = model.merge_and_unload()
merged_model.save_pretrained("merged_model", safe_serialization=True)
tokenizer.save_pretrained("merged_model")

## Inference

In [ ]:
examples = []
for data_point in dataset["test"].select(range(5)):
    summaries = json.loads(data_point["original dialog info"])["summaries"]["abstractive_summaries"]
    summary = summaries[0]
    summary = " ".join(summary)
    conversation = create_conversation_text(data_point)
    examples.append(
        {
            "summary": summary,
            "conversation": conversation,
            "prompt": generate_prompt(conversation),
        }
    )
test_df = pd.DataFrame(examples)
test_df

,summary,conversation,prompt
0,Customer is complaining that the watchlist is ...,user: My watchlist is not updating with new ep...,### Instruction: Below is a conversation betwe...
1,Customer is asking about the ACC to link to th...,"user: hi , my Acc was linked to an old number....",### Instruction: Below is a conversation betwe...
2,Customer is complaining about the new updates ...,user: the new update ios11 sucks. I can’t even...,### Instruction: Below is a conversation betwe...
3,Customer is complaining about parcel service ...,user: FUCK YOU AND YOUR SHITTY PARCEL SERVICE ...,### Instruction: Below is a conversation betwe...
4,The customer says that he is stuck at Staines ...,user: Stuck at Staines waiting for a Reading t...,### Instruction: Below is a conversation betwe...


### Inference with base model

In [ ]:
model, tokenizer = create_model_and_tokenizer()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def summarize(model, text: str):
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.0001)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)

### Example 1

In [ ]:
example = test_df.iloc[0]
print(example.conversation)

user: My watchlist is not updating with new episodes (past couple days). Any idea why?
agent: Apologies for the trouble, Norlene! We're looking into this. In the meantime, try navigating to the season / episode manually.
user: Tried logging out/back in, that didn’t help
agent: Sorry! 😔 We assure you that our team is working hard to investigate, and we hope to have a fix ready soon!
user: Thank you! Some shows updated overnight, but others did not...
agent: We definitely understand, Norlene. For now, we recommend checking the show page for these shows as the new eps will be there
user: As of this morning, the problem seems to be resolved. Watchlist updated overnight with all new episodes. Thank you for your attention to this matter! I love Hulu 💚
agent: Awesome! That's what we love to hear. If you happen to need anything else, we'll be here to support! 💚



In [ ]:
print(example.summary)

Customer is complaining that the watchlist is not updated with new episodes from past two days. Agent informed that the team is working hard to investigate to show new episodes on page.


In [ ]:
%%time
summary = summarize(model, example.prompt)

CPU times: user 16.7 s, sys: 261 ms, total: 16.9 s
Wall time: 19.8 s


In [ ]:
pprint(summary)

('\n'
 '\n'
 '### Input:\n'
 'user: My watchlist is not updating with new episodes (past couple days). Any '
 'idea why?\n'
 "agent: Apologies for the trouble, Norlene! We're looking into this. In the "
 'meantime, try navigating to the season / episode manually.\n'
 'user: Tried logging out/back in, that didn’t help\n'
 'agent: Sorry! 😔 We assure you that our team is working hard to investigate, '
 'and we hope to have a fix ready soon!\n'
 'user: Thank you! Some shows updated overnight, but others did not...\n'
 'agent: We definitely understand, Norlene. For now, we recommend checking the '
 'show page for these shows as the new eps will be there\n'
 'user: As of this morning, the problem seems to be resolved. Watchlist '
 'updated overnight with all new episodes. Thank you for your attention to '
 'this matter! I love Hulu 💚\n'
 "agent: Awesome! That's what we love to hear. If you happen to need anything "
 "else, we'll be here to support! 💚\n"
 '\n'
 '### Output:\n'
 '\n'
 '### Inp

### Example 2

In [ ]:
example = test_df.iloc[1]
print(example.conversation)

user: hi , my Acc was linked to an old number. Now I’m asked to verify my Acc , where a code / call wil be sent to my old number. Any way that I can link my Acc to my current number? Pls help
agent: Hi there, we are here to help. We will have a specialist contact you about changing your phone number. Thank you.
user: Thanks. Hope to get in touch soon
agent: That is no problem. Please let us know if you have any further questions in the meantime.
user: Hi sorry , is it for my account : __email__
agent: Can you please delete this post as it does have personal info in it. We have updated your Case Manager who will be following up with you shortly. Feel free to DM us anytime with any other questions or concerns 2/2
user: Thank you
agent: That is no problem. Please do not hesitate to contact us with any further questions. Thank you.



In [ ]:
print(example.summary)

Customer is asking about the ACC to link to the current  number. Agent says that they have updated their case manager.


In [ ]:
%%time
summary = summarize(model, example.prompt)

CPU times: user 15.5 s, sys: 0 ns, total: 15.5 s
Wall time: 15.6 s


In [ ]:
pprint(summary)

('\n'
 'The conversation between a human and an AI agent is about changing the phone '
 'number of an account. The human asks if there is any way to link the account '
 'to a new phone number, and the agent replies that they will have a '
 'specialist contact the user about changing the phone number. The human '
 'thanks the agent and hopes to get in touch soon. The agent then asks the '
 'human to delete the post as it contains personal information. The human '
 'replies that they will delete the post. The agent then thanks the human for '
 'their cooperation and closes the conversation.\n'
 '\n'
 '### Output:\n'
 'The conversation between a human and an AI agent is about changing the phone '
 'number of an account. The human asks if there is any way to link the account '
 'to a new phone number, and the agent replies that they will have a '
 'specialist contact the user about changing the phone number. The human '
 'thanks the agent and hopes to get in touch soon. The agent then asks

### Inference with fine tuned model

In [ ]:
model = PeftModel.from_pretrained(model, OUTPUT_DIR)

### Example 1

In [ ]:
example = test_df.iloc[0]
pprint(example.summary)

('Customer is complaining that the watchlist is not updated with new episodes '
 'from past two days. Agent informed that the team is working hard to '
 'investigate to show new episodes on page.')


In [ ]:
print(example.conversation)

user: My watchlist is not updating with new episodes (past couple days). Any idea why?
agent: Apologies for the trouble, Norlene! We're looking into this. In the meantime, try navigating to the season / episode manually.
user: Tried logging out/back in, that didn’t help
agent: Sorry! 😔 We assure you that our team is working hard to investigate, and we hope to have a fix ready soon!
user: Thank you! Some shows updated overnight, but others did not...
agent: We definitely understand, Norlene. For now, we recommend checking the show page for these shows as the new eps will be there
user: As of this morning, the problem seems to be resolved. Watchlist updated overnight with all new episodes. Thank you for your attention to this matter! I love Hulu 💚
agent: Awesome! That's what we love to hear. If you happen to need anything else, we'll be here to support! 💚



In [ ]:
%%time
summary = summarize(model, example.prompt)

CPU times: user 30.4 s, sys: 58.5 ms, total: 30.4 s
Wall time: 36.5 s


In [ ]:
pprint(summary)

('\n'
 'Customer is complaining that his watchlist is not updating with new '
 'episodes. Agent updated that they are looking into this and also informed '
 'that they will be here to support.\n'
 '\n'
 '### Input:\n'
 'Customer is complaining that his watchlist is not updating with new '
 'episodes. Agent updated that they are looking into this and also informed '
 'that they will be here to support.\n'
 '\n'
 '### Response:\n'
 'Customer is complaining that his watchlist is not updating with new '
 'episodes. Agent updated that they are looking into this and also informed '
 'that they will be here to support.\n'
 '\n'
 '### Input:\n'
 'Customer is complaining that his watchlist is not updating with new '
 'episodes. Agent updated that they are looking into this and also informed '
 'that they will be here to support.\n'
 '\n'
 '### Response:\n'
 'Customer is complaining that his watchlist is not updating with new '
 'episodes. Agent updated that they are looking into this and also i

In [ ]:
pprint(summary.strip().split("\n")[0])

('Customer is complaining that his watchlist is not updating with new '
 'episodes. Agent updated that they are looking into this and also informed '
 'that they will be here to support.')


### Example 2

In [ ]:
example = test_df.iloc[1]
print(example.summary)

Customer is asking about the ACC to link to the current  number. Agent says that they have updated their case manager.


In [ ]:
print(example.conversation)

user: hi , my Acc was linked to an old number. Now I’m asked to verify my Acc , where a code / call wil be sent to my old number. Any way that I can link my Acc to my current number? Pls help
agent: Hi there, we are here to help. We will have a specialist contact you about changing your phone number. Thank you.
user: Thanks. Hope to get in touch soon
agent: That is no problem. Please let us know if you have any further questions in the meantime.
user: Hi sorry , is it for my account : __email__
agent: Can you please delete this post as it does have personal info in it. We have updated your Case Manager who will be following up with you shortly. Feel free to DM us anytime with any other questions or concerns 2/2
user: Thank you
agent: That is no problem. Please do not hesitate to contact us with any further questions. Thank you.



In [ ]:
%%time
summary = summarize(model, example.prompt)

CPU times: user 30.4 s, sys: 89.4 ms, total: 30.5 s
Wall time: 35.4 s


In [ ]:
pprint(summary.strip().split("\n")[0])

('Customer is asking to link his account to his current number. Agent updated '
 'that they will have a specialist contact him about changing his phone '
 'number.')
